#Getting started
Let's be sure that we are in our working directory . . .

In [ ]:
cd ~/bioinf_1

In [ ]:
pwd

#Mapping with Tophat

## Tophat Flowchart
http://www.genomebiology.com/2013/14/4/R36/figure/F6

## Reference Genome and Annotation
We have some preparation to do before we can map our data.  First we need to download a reference genome and its annotation file.  It is very important *that the genome sequence and annotation are the same version*, if they are not, things could go horribly wrong.  The best way to ensure that your sequence and annotation are compatible is to download both from the same place, at the same time, and double check that they have the same version number.  There are several good places to get genomes data:

* Illumina's website http://support.illumina.com/sequencing/sequencing_software/igenome.html.
* NCBI http://www.ncbi.nlm.nih.gov
* Ensembl http://www.ensembl.org/info/about/species.html

Illumina has several *Escherichia coli* genomes, but not the one we are working with, which is *E. coli* K-12 strain W3110.

### NCBI
NCBI has most published genomes, but it is a bit tricky to find exactly what we are looking for.  Let's start at the NCBI Genome Assembly page http://www.ncbi.nlm.nih.gov/assembly/ and search for "Escherichia coli W3110".  This gets us to http://www.ncbi.nlm.nih.gov/assembly/GCF_000010245.2/, which is part way there.  On the right side of the W3110 Assembly page is a link for "GenBank FTP site":  ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000010245.1_ASM1024v1.  This is exactly what we want (note: Firefox can handle FTP, but Safari cannot)!  From here we want two files:

* The genome sequence GCA_000010245.1_ASM1024v1_genomic.fna.gz (.fna is short for FASTA nucleic acid, .gz means its gzipped) - ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000010245.1_ASM1024v1/GCA_000010245.1_ASM1024v1_genomic.fna.gz
* The genome annotation GCA_000010245.1_ASM1024v1_genomic.gff.gz (.gff is Generic Feature Format) ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000010245.1_ASM1024v1/GCA_000010245.1_ASM1024v1_genomic.gff.gz

### Ensembl
Ensembl is similarly difficult to navigate.  We will start at the [Ensembl Bacteria](http://bacteria.ensembl.org/index.html) page, and again search for "Escherichia coli W3110" in the *Search for a genome* search box.  This will get us to the [Escherichia coli str. K-12 substr. W3110](http://bacteria.ensembl.org/escherichia_coli_str_k_12_substr_w3110/Info/Index) page. On the right side, are links for [FASTA](ftp://ftp.ensemblgenomes.org/pub/bacteria/release-28/fasta/bacteria_23_collection/escherichia_coli_str_k_12_substr_w3110) and [GFF3](ftp://ftp.ensemblgenomes.org/pub/bacteria/release-28/gff3/bacteria_23_collection/escherichia_coli_str_k_12_substr_w3110).  There are several options for the genome sequence, but the one we want is the [complete unmasked assembled sequence](ftp://ftp.ensemblgenomes.org/pub/bacteria/release-28/fasta/bacteria_23_collection/escherichia_coli_str_k_12_substr_w3110/dna/Escherichia_coli_str_k_12_substr_w3110.GCA_000010245.1.28.dna.genome.fa.gz).

### Downloading with wget
Now we can use the `wget` command to actually download these files. We will get the files from NCBI. Here is what we will want to tell wget:

* --output-document : the name to use when saving the file
* --no-verbose : don't output a lot of information while downloading
* URL : what to download

We are going to make a "genome" directory for these files so that things don't get too messy.  Soon we will generate several files based on these that tophat needs.  I generally like to keep the original file names, but we are changing the names to make typing easier later.  We are changing the FASTA file ending from ".fna" to ".fa", because tophat wants a file names ".fa", its picky that way.

In [ ]:
%%bash
mkdir -p genome
wget --output-document genome/ecoli_w3110.fa.gz --no-verbose ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000010245.1_ASM1024v1/GCA_000010245.1_ASM1024v1_genomic.fna.gz
wget --output-document genome/ecoli_w3110.gff.gz --no-verbose ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000010245.1_ASM1024v1/GCA_000010245.1_ASM1024v1_genomic.gff.gz
ls genome

### Decompressing the reference files
Now we need to decompress the FASTA and GFF files.  We will use gunzip

In [ ]:
%%bash
gunzip genome/ecoli_w3110.fa.gz
gunzip genome/ecoli_w3110.gff.gz
ls genome

Let's take a quick look at these files.  The `head` command shows us the first few lines of a file (default is 10).

In [ ]:
%%bash
head genome/ecoli_w3110.fa

In [ ]:
%%bash
head genome/ecoli_w3110.gff

Unfortunately the GFF file has long lines, which are wrapping onto the next line, making them hard to read.  Another option is to use the command `less -S genome/ecoli_w3110.gff` in the terminal.  The "-S" says to truncate lines instead of wrapping them.

#### GFF, a brief aside
You can find one description of the GFF format here http://www.sequenceontology.org/gff3.shtml.  Unfortunately it is not entirely standard, there are several different version numbers (1, 2, 2.5, 3), and some variations within these version numbers.  By getting our annotation from NCBI, we have a good chance that the GFF format will be compatible with most software.

### Indexing the Genome
Before we can map reads to the reference genome using Bowtie or Tophat, we need to index it. This will generate a transformed version of the genome that allows Bowtie to efficiently map sequences to it. We use bowtie2-build (part of the Bowtie package) to do this. The command for bowtie2-build is `bowtie2-build REF_GENOME INDEX_PATH`. 

* REF_GENOME : the file containing the reference sequence, this must be in FASTA format.
* INDEX_PATH : The names of the index files generated by bowtie2-build will all start with INDEX_PATH, and when actually run Bowtie, we will also supply it with INDEX_PATH, and it will find all the files. Note: INDEX_PATH can be anything we want, it does not need to be at related to the name of the REF_GENOME file, but it does make things less confusing if we are consistent.

So here is how we run bowtie2-build: 

In [ ]:
%%bash
bowtie2-build genome/ecoli_w3110.fa genome/ecoli_w3110

Now let's check to be sure that worked:

In [ ]:
ls genome

## Mapping with Tophat

We will use Tophat to map the RNA-Seq reads. Tophat runs bowtie, but with a twist - it uses the GTF file and the genome sequence to generate a virtual transcriptome. It tries to map each read to the transcriptome, then to the genome, then it tries to identify novel splice sites that could have resulted in the read. This is explained in this [flowchart](http://genomebiology.com/2013/14/4/R36/figure/F6) from the [Tophat2 publication](http://dx.doi.org/10.1186/gb-2013-14-4-r36)

### Running Tophat

First lets set up a directory for the Tophat output, to keep things tidy.

In [ ]:
%%bash
mkdir -p th_out
ls

The minimum tophat command is `tophat2 INDEX_PATH FASTQ`. This is the same `INDEX_PATH` that we used for bowtie2-build. Although that command is enough to run Tophat, it is going to be very useful to give it some more options: 

* -G GTF_FILE : Tophat uses the genome annotation to figure out where exons are (i.e. where it expects to see mRNA). 
* –library-type LIBRARY_TYPE : This tells tophat whether the data is stranded, and if so, which strand was sequenced.
* –output-dir DIRECTORY : where to put the results
* –max-intron-length NUM_BP : Tophat uses intron size defaults that are optimized for human genomes, these values are unreasonable for a microbial genome (of course we don't expect any introns in *E. coli*).
* –min-intron-length NUM_BP : also optimized for human genomes, and unreasonable for a microbial genome.
* --transcriptome-max-hits NUM_HITS : Maximum number of mappings allowed for a read, when aligned to the transcriptome (any reads found with more then this number of mappings will be discarded). 
* --max-multihits NUM_HITS : number of alignments to report if there is more than one.
* --no-coverage-search : turns off searching for novel splice junctions based on read depth
* --no-novel-juncs : do not try to find novel splice junctions
* --num-threads NUM : number of cpus to use

We will start with these parameters, but there is an extensive list of  command line options detailed in the [Tophat Manual](https://ccb.jhu.edu/software/tophat/manual.shtml), it is a good idea to read through and try to understand all of them.  We will discuss some more later.

#### Running Tophat for one sample
Now, for each sample we need to make a directory for the tophat output, then run tophat on that sample. 

In [ ]:
%%bash
mkdir -p th_out/r1_test
ls th_out

In [ ]:
%%bash
tophat2 -G genome/ecoli_w3110.gff \
    --library-type fr-firststrand \
    --output-dir th_out/r1_test \
    --max-intron-length 5 \
    --min-intron-length 4 \
    --transcriptome-max-hits 1 \
    --max-multihits 1 \
    --no-coverage-search \
    --no-novel-juncs \
    --num-threads 2 \
    genome/ecoli_w3110 \
    r1.test.trim.fastq.gz

### Tophat Output
So what happened? Let's take a look . . .

In [ ]:
ls th_out/r1_test

Tophat generates a lot of files. Let's focus on these:

    accepted_hits.bam : this is the mapped reads
    align_summary.txt : summary of mapping results
    unmapped.bam : the unmapped reads

In [ ]:
%%bash
head th_out/r1_test/align_summary.txt

accepted_hits.bam and unmapped.bam are both BAM files.  We can look at them using a program called samtools.  This is probably better to do in the terminal, but we will take a crack at it here:

In [ ]:
%%bash
samtools view th_out/r1_test/accepted_hits.bam | head